In [1]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
EPOCHS = 35
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((100,100))
image_size = 0
directory_root = 'data'
width=100
height=100
depth=3

Function to convert images to array

In [3]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

Fetch images from directory

In [4]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:250]:
                image_directory = f"{directory_root}/{plant_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing Corn_Blight (1).jpeg ...
[INFO] Processing Corn_Blight (1).jpg ...
[INFO] Processing Corn_Blight (10).jpg ...
[INFO] Processing Corn_Blight (100).jpg ...
[INFO] Processing Corn_Blight (101).jpg ...
[INFO] Processing Corn_Blight (102).jpg ...
[INFO] Processing Corn_Blight (103).jpg ...
[INFO] Processing Corn_Blight (104).jpg ...
[INFO] Processing Corn_Blight (105).jpg ...
[INFO] Processing Corn_Blight (106).jpg ...
[INFO] Processing Corn_Blight (107).jpg ...
[INFO] Processing Corn_Blight (108).jpg ...
[INFO] Processing Corn_Blight (109).jpg ...
[INFO] Processing Corn_Blight (11).jpg ...
[INFO] Processing Corn_Blight (110).jpg ...
[INFO] Processing Corn_Blight (111).JPG ...
[INFO] Processing Corn_Blight (112).JPG ...
[INFO] Processing Corn_Blight (113).JPG ...
[INFO] Processing Corn_Blight (114).JPG ...
[INFO] Processing Corn_Blight (115).JPG ...
[INFO] Processing Corn_Blight (116).JPG ...
[INFO] Processing Corn_Blight (117).JPG ...
[INFO] Proc

Get Size of Processed Image

In [5]:
image_size = len(image_list)

Transform Image Labels uisng [Scikit Learn](http://scikit-learn.org/)'s LabelBinarizer

In [6]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

Print the classes

In [7]:
print(label_binarizer.classes_)

['Corn_Blight (1).jpeg' 'Corn_Blight (1).jpg' 'Corn_Blight (10).jpg'
 'Corn_Blight (100).jpg' 'Corn_Blight (101).jpg' 'Corn_Blight (102).jpg'
 'Corn_Blight (103).jpg' 'Corn_Blight (104).jpg' 'Corn_Blight (105).jpg'
 'Corn_Blight (106).jpg' 'Corn_Blight (107).jpg' 'Corn_Blight (108).jpg'
 'Corn_Blight (109).jpg' 'Corn_Blight (11).jpg' 'Corn_Blight (110).jpg'
 'Corn_Blight (111).JPG' 'Corn_Blight (112).JPG' 'Corn_Blight (113).JPG'
 'Corn_Blight (114).JPG' 'Corn_Blight (115).JPG' 'Corn_Blight (116).JPG'
 'Corn_Blight (117).JPG' 'Corn_Blight (118).JPG' 'Corn_Blight (119).JPG'
 'Corn_Blight (12).jpg' 'Corn_Blight (13).jpg' 'Corn_Blight (14).jpg'
 'Corn_Blight (15).jpg' 'Corn_Blight (16).jpg' 'Corn_Blight (17).jpg'
 'Corn_Blight (18).jpg' 'Corn_Blight (19).jpg' 'Corn_Blight (2).jpg'
 'Corn_Blight (20).jpg' 'Corn_Blight (21).jpg' 'Corn_Blight (22).jpg'
 'Corn_Blight (23).jpg' 'Corn_Blight (24).jpg' 'Corn_Blight (25).jpg'
 'Corn_Blight (26).jpg' 'Corn_Blight (27).jpg' 'Corn_Blight (28).jpg'
 '

In [8]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [9]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

[INFO] Spliting data to train, test


In [10]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [11]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

Model Summary

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 32)      896       
                                                                 
 activation (Activation)     (None, 100, 100, 32)      0         
                                                                 
 batch_normalization (BatchN  (None, 100, 100, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 33, 33, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 33, 33, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 33, 33, 64)        1

In [13]:
#Compiling Model
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

[INFO] training network...


C:\Users\AAYUSH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
#Fitting Model
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

C:\Users\AAYUSH\AppData\Local\Temp\ipykernel_2720\2474006815.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/35
1109/1109 [==============================] - 762s 683ms/step - loss: 0.0687 - accuracy: 0.0039 - val_loss: 0.0194 - val_accuracy: 0.0042
Epoch 2/35
 336/1109 [========>.....................] - ETA: 8:13 - loss: 0.0193 - accuracy: 0.0055

KeyboardInterrupt: 

Model Accuracy

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
54/54 [==============================] - 7s 119ms/step - loss: 0.0353 - accuracy: 0.0093
Test Accuracy: 0.9259259328246117


In [ ]:
# Save the model using TensorFlow SavedModel format
model.save('my_model.h5')

In [ ]:
scores = model.evaluate(x_train, y_train)
print(f"Test Accuracy: {scores[1]*100}")

216/216 [==============================] - 26s 120ms/step - loss: 0.0342 - accuracy: 0.0126
Test Accuracy: 1.25868059694767
